In [33]:
import numpy as np
from PIL import Image
import glob
import matplotlib.pyplot as plt

In [34]:
def iterative_flood_fill(image, x, y, visited):
    stack = [(x, y)]  # Initialize stack with the starting pixel

    while stack:
        x, y = stack.pop()

        if x < 0 or x >= image.shape[1] or y < 0 or y >= image.shape[0]:
            continue  # Skip out-of-bounds pixels
        if image[y, x] == 1 or visited[y, x]:
            continue  # Skip edge pixels and already visited pixels

        visited[y, x] = True  # Mark the pixel as visited

        # Add the neighboring pixels to the stack
        stack.append((x + 1, y))  # Right
        stack.append((x - 1, y))  # Left
        stack.append((x, y + 1))  # Down
        stack.append((x, y - 1))  # Up


def outer_edge_detector_iterative(image):
    visited = np.zeros_like(image)  # Initialize the visited array

    # Apply flood fill to the boundary pixels of the image
    for i in range(image.shape[0]):
        iterative_flood_fill(image, 0, i, visited)  # Left edge
        iterative_flood_fill(image, image.shape[1] - 1, i, visited)  # Right edge

    for j in range(image.shape[1]):
        iterative_flood_fill(image, j, 0, visited)  # Top edge
        iterative_flood_fill(image, j, image.shape[0] - 1, visited)  # Bottom edge

    return visited

In [35]:
image_files = glob.glob("imgs_epoch_019/*.png")

for image in image_files:
    i = Image.open(image)
    i_rgb = np.array(i.convert("RGB"))

    i = np.array(i)
    i_filtered = i > 50

    mask = outer_edge_detector_iterative(i_filtered)
    i_rgb_r = i_rgb[..., 0]
    i_rgb_g = i_rgb[..., 1]
    i_rgb_b = i_rgb[..., 2]
    i_rgb_r[(~mask) & (i_filtered == 0)] = 255

    saved = Image.fromarray(i_rgb)
    saved.save(image)

KeyboardInterrupt: 